# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

### Imports

In [24]:
import pandas as pd
import numpy as np
from mpmath import *

In [25]:
leitura = pd.ExcelFile('tweets_Nike_1.xlsx')

In [26]:
tweets_treinamento = pd.read_excel(leitura, 'Treinamento')

In [27]:
tweets_teste = pd.read_excel(leitura, 'Teste') 

In [110]:
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace(':', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('@', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('*', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('“', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('.', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace(')', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('(', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('_', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace(';', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace(',', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('&', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('/', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('#', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('%', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('$', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace("!", '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace("?", '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace("-", '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace("+", '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace("=", '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('rt', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace("\n", '', regex=False)

In [111]:
tweets_teste.Teste = tweets_teste.Teste.str.replace(':', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('@', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('*', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('“', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('.', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace(')', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('(', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('_', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace(';', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace(',', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('&', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('/', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('#', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('%', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('$', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace("!", '')
tweets_teste.Teste = tweets_teste.Teste.str.replace("?", '')
tweets_teste.Teste = tweets_teste.Teste.str.replace("-", '')
tweets_teste.Teste = tweets_teste.Teste.str.replace("+", '')
tweets_teste.Teste = tweets_teste.Teste.str.replace("=", '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('rt', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace("\n", '')

In [112]:
rel = tweets_treinamento[tweets_treinamento['classificacao'] == 1]
irrel = tweets_treinamento[tweets_treinamento['classificacao'] == 0]
tweets_treinamento['Treinamento'] = tweets_treinamento

In [113]:
lrel= []
for i in tweets_treinamento['Treinamento']:
    lrel.append(rel['Treinamento'])

In [114]:
lirrel= []
for i in tweets_treinamento['Treinamento']:
    lirrel.append(irrel['Treinamento'])

In [115]:
n_irrel = tweets_treinamento.classificacao[tweets_treinamento.classificacao == 0].count()
n_rel = tweets_treinamento.classificacao[tweets_treinamento.classificacao == 1].count()
total = n_rel + n_irrel

prob_irrelevante = n_irrel/ total
prob_relevante = n_rel/ total

tweets = []
relevante = []
irrelevante = []

for palavra in tweets_treinamento['Treinamento']:
    palavra = palavra.lower()
    
for palavra in rel['Treinamento']:
    relevante.append(palavra.split())
    
for palavra in irrel['Treinamento']:
    irrelevante.append(palavra.split())

for each in tweets_treinamento['Treinamento']:
    tweets.append(each.split())

dic_rel = {}
dic_irrel = {}
dic_tweet = {}

Ln_rel = []
Ln_irrel = []
TD = []

for f in relevante:
    for p in f:
        Ln_rel.append(p)
        dic_rel[word] = Ln_rel.count(p)

for f in irrelevante:
    for p in f:
        Ln_irrel.append(p)
        dic_irrel[p] = Ln_irrel.count(p)

for f in tweets:
    for p in f:
        TD.append(p)
        dic_tweet[p] = TD.count(p)

In [116]:
P_relevante = len(rel)/len(tweets_treinamento)
P_irrelevante = len(irrel)/len(tweets_treinamento)

In [117]:
print(len(TD))
print(len(Ln_rel))
print(len(Ln_irrel))

5378
377
5001


In [118]:
new= list(set(TD))
print(len(new))
print(len(TD))

1592
5378


In [119]:
r = len(Ln_rel)/len(TD)
ir = len(Ln_irrel)/len(TD)

In [120]:
p_r = {}
p_ir = {}

for pl in Ln_rel:
    p_r[pl] = (dic_rel[pl] + 1)/ (len(new) + len(Ln_rel))

for pl in Ln_irrel:
    p_ir[pl] = (dic_irrel[pl] + 1)/ (len(new) + len(Ln_irrel))


KeyError: 'nike'

In [121]:
p_r

{}

In [122]:
soma_rel = sum(p_r.values())
soma_irrel = sum(p_ir.values())

print(soma_rel + soma_irrel)

0


In [123]:
def CalculaProbR(tweet):
    a = 1
    T_tweet = 0
    
    for frase in relevantes_separados:
        T_tweet += len(frase)
        
        for j in frase:
            if j.lower() == tweet.lower():
                a += 1
                
    return a/(T_tweet + len(repeticoes))

In [124]:
def CalculaProbI(tweet):
    a = 1
    T_tweet = 0
    
    for frase in irrelevantes_separados:
        T_tweet += len(frase)
        
        for j in frase:
            if j.lower() == tweet.lower():
                a += 1
                
    return a/(T_tweet + len(repeticoes))

In [125]:
def verifica(td_separado):
    varifica_pronto = []

    for t in td_separado: 
        
        td_s = t.split()
        
        Prob_relevantes = []
        Prob_irrelevantes = []
                
        for p in td_s:
                        
            Prob_relevantes.append(CalculaProbR(p))
            Prob_irrelevantes.append(CalculaProbI(p))

        relevante = 1
        irrelevante = 1

        for i in Prob_relevantes:
            relevante *= i

        for i in Prob_irrelevantes:
            irrelevante *= i

        relevante_tweet = relevante * Prob_relevantes
        irrelevante_tweet = irrelevante * Prob_irrelevantes

        if relevate > irrelevante:
            verifica_pronto.append('Relevante')
        else:
            verifica_pronto.append('Irrelevante')
    return varifica_pronto

Ltweets_test = []
for tweet in tweets_teste:
    Ltweets_test.append(tweet)

verifica_pronto = varifica(Ltweets_test)

resposta = tweets_teste.rename(index=str, columns={'Unnamed: 1':'Relevancia'}).Relevancia

NameError: name 'varifica' is not defined

In [126]:
falpos = 0
falneg = 0
verpos = 0
verneg = 0

for i in range(len(resposta)):
    if resposta[i] == verifica_pronto[i] and resposta[i] == 'Relevante':
        verpos += 1
    elif resposta[i] == verifica_pronto[i] and resposta[i] == 'Irrelevante':
        verneg += 1
    elif resposta[i] != verifica_pronto[i] and resposta[i] == 'Relevante':
        falneg += 1
    else:
        falpos += 1

falpos /= len(verifica_pronto)
falneg /= len(verifica_pronto)
verpos /= len(verifica_pronto)
verneg /= len(verifica_pronto)


print("Negativo Falso: {}%".format(falneg*100))
print("Positivo Falso: {}%".format(falpos*100))
print("Negativo Verdadeiro: {}%".format(verneg*100))
print("Positivo Verdadeiro: {}%".format(verpos*100))

len(verifica_pronto)

NameError: name 'resposta' is not defined

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
